In [20]:
import pandas as pd
import seaborn as sns
import pyodbc
from gensim import corpora, models, similarities
import nltk
import re
import random
import binascii
import logging, gensim, bz2
from nltk.tag import pos_tag
import logging
import re
import tempfile
import tarfile, zipfile
from gensim.utils import tokenize
from collections import defaultdict
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import vertica_python

tenant_id = 289
unuseful_list = 'for a of the and to in is very i my on it was as he she they us this hello hi over';
#search_comment = "";

frequency = defaultdict(int)

conn_info = {'host': 'vertica04.reputation.ec2', 
             'port': 5433, 
             'user': 'rep_engine_user', 
             'password': 'r3p_3ng1n3', 
             'database': 'rwarehouse'}

cnxn = vertica_python.connect(**conn_info)

%matplotlib inline
sample_size = 10000

In [21]:
query= """
SELECT id, comment_title FROM r4e_mongo.ratings 
WHERE 
    industry in ('health-care-physicians') 
AND comment_title <>''
"""

result = pd.read_sql(query, cnxn)

result

,id,comment_title
0,BOOKING.SCafB2F6rCcLiJwIy6J_tQ.-NlPc3tSDK9K8_Y...,Good
1,BOOKING.SCafB2F6rCcLiJwIy6J_tQ.0UbpUlPrwVpW2Jz...,“Old and not clean.”
2,BOOKING.SCafB2F6rCcLiJwIy6J_tQ.13t5MTtLjvNwDil...,“Schrecklich”
3,BOOKING.SCafB2F6rCcLiJwIy6J_tQ.1_4t1AWear2pY8Q...,Very poor
4,BOOKING.SCafB2F6rCcLiJwIy6J_tQ.2yqwk3ddl9jcUTr...,“Fair at best”
5,BOOKING.SCafB2F6rCcLiJwIy6J_tQ.3rvvGPlm6x7UZzi...,Good
6,BOOKING.SCafB2F6rCcLiJwIy6J_tQ.5D_uIrrKV8L8zfS...,“Disappointed”
7,BOOKING.SCafB2F6rCcLiJwIy6J_tQ.5FekO4omUtNi-uN...,Exceptional
8,BOOKING.SCafB2F6rCcLiJwIy6J_tQ.5KW7O-H-MxODbZI...,Pleasant
9,BOOKING.SCafB2F6rCcLiJwIy6J_tQ.68Lhra-BpXEMSgD...,“Great place to stay but lacks an elevator for...


In [22]:
# the labeled training data
import pandas as pd       
df = pd.read_csv("physicianTraining.csv")

from sklearn.cross_validation import train_test_split
topics, testTopics = train_test_split(df, test_size=0.35, random_state=42)

In [23]:
print topics.size

print testTopics.size

266760
143640


In [24]:
topics = topics.drop_duplicates(subset='comment')
topics = topics.dropna(subset=['comment'])
topics = topics.drop('comment', 1)

topics = topics.reset_index()

testTopics = testTopics.drop_duplicates(subset='comment')
testTopics = testTopics.dropna(subset=['comment'])
testTopics = testTopics.drop('comment', 1)

testTopics = testTopics.reset_index()

In [25]:
#print topics.columns.values
#print result.columns.values

train = pd.merge(topics, result, on='id');

train = train.rename(columns={'comment_title':'comment'})
#print train.columns.values

train = train.dropna(subset=['what_topics_is_this_review_about'])

train = train.drop_duplicates(subset='comment')
train = train.dropna(subset=['comment'])

train = train.reset_index()

#print(train)
print train['comment']
print train['what_topics_is_this_review_about']

from sklearn.preprocessing import MultiLabelBinarizer
binarizer = MultiLabelBinarizer()

test_target = binarizer.fit_transform(train.what_topics_is_this_review_about)
print test_target

finalTrain = train



testTrain = pd.merge(testTopics, result, on='id');

testTrain = testTrain.rename(columns={'comment_title':'comment'})
#print testTrain.columns.values

testTrain = testTrain.dropna(subset=['what_topics_is_this_review_about'])

testTrain = testTrain.drop_duplicates(subset='comment')
testTrain = testTrain.dropna(subset=['comment'])

testTrain = testTrain.reset_index()
#print(testTrain)
print testTrain['comment']
print testTrain['what_topics_is_this_review_about']

finalTest = testTrain

0                                               Agressive
1                                 Needs Lesson in Manners
2                    Thorough with a great bedside manner
3                               Long Wait for Colonoscopy
4                                                     Dr.
5                                        too smug by half
6                                            Great doctor
7                                             Poor Review
8                         very strange and unprofessional
9                               He should probably retire
10                                             Medication
11      Medical Malpractice That Could Lead To Death I...
12                               Avoid Like The Plague!!!
13                     Why has he done so many surgeries?
14           Negative, Arrogant 3-Minute Consultation!!!!
15                                Very rude pediatrician!
16               The most thorough doctor I have ever met
17            

In [26]:
print finalTrain.size

print finalTest.size

22484
19250


In [27]:
#import gensim
#from gensim import utils
#from gensim.models import Phrases

#print train['comment'][0]

#num_reviews = train['comment'].size

#for i in xrange( 0, num_reviews ):
#    train['comment'][i] = utils.simple_preprocess(train['comment'][i])

#bigram = Phrases(train['comment'])

#print train['comment'][0]
#print bigram[train['comment'][0]]

#print testTrain['comment'][0]

#num_reviews = testTrain['comment'].size

#for i in xrange( 0, num_reviews ):
#    testTrain['comment'][i] = utils.simple_preprocess(testTrain['comment'][i])

#bigram = Phrases(testTrain['comment'])

#print testTrain['comment'][0]
#print bigram[testTrain['comment'][0]]

In [28]:
#num_reviews = train["comment"].size

#for i in xrange( 0, num_reviews ):
#    train['comment'][i] = ' '.join(train['comment'][i])
    
#print train['comment'][900]

#num_reviews = testTrain["comment"].size

#for i in xrange( 0, num_reviews ):
#    testTrain['comment'][i] = ' '.join(testTrain['comment'][i])
    
#print testTrain['comment'][10]

In [29]:
import re
import nltk
#nltk.download()  # Download text data sets, including stop words

In [30]:
from nltk.corpus import stopwords # Import the stop word list
print stopwords.words("english") 

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

In [31]:
def review_to_words( raw_review ):

    letters_only = re.sub("[^a-zA-Z]", " ", raw_review) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))  


In [32]:
# Get the number of reviews based on the dataframe column size
num_reviews = train['comment'].size

print "Parsing training set...\n"
clean_train_reviews = []
for i in xrange( 0, num_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Comment %d of %d\n" % ( i+1, num_reviews )
    print train["comment"][i]
    clean_train_reviews.append( review_to_words( train["comment"][i] ))
    
num_reviews = testTrain['comment'].size

print "Parsing testTraining set...\n"
clean_testTrain_reviews = []
for i in xrange( 0, num_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Comment %d of %d\n" % ( i+1, num_reviews )                                                                    
    clean_testTrain_reviews.append( review_to_words( testTrain["comment"][i] ))
    
print "Completed"

Parsing training set...

Agressive
Needs Lesson in Manners
Thorough with a great bedside manner
Long Wait for Colonoscopy
Dr.
too smug by half
Great doctor
Poor Review
very strange and unprofessional
He should probably retire
Medication
Medical Malpractice That Could Lead To Death In A Patient!
Avoid Like The Plague!!!
Why has he done so many surgeries?
Negative, Arrogant 3-Minute Consultation!!!!
Very rude pediatrician!
The most thorough doctor I have ever met
Parent of Janicea Wilson
weirdest appointment that i have ever had
Proceed with caution
Horrible!
Poor physician
Worst doctor I've ever had the mispleasure
breast  explant
I do NOT recommend!
he was my ob
Good experience.
Great Doctor
Do not waste your time with this Doctor
Best Doctor, Best Staff
Staff Blows
Mrs
This is a horrible, arrogant physician. Avoid him in every instance.
Disturbed, Dazed & Confused
I do not recommend Dr. Takami
See another doctor
No answers
Dr. Zweiban
OWNER OF NXT LEVEL NEIWORKING
Really poor attitude

In [33]:
clean_train_reviews

clean_testTrain_reviews

[u'good doctor',
 u'needs much improvement',
 u'great doctor',
 u'dr melissa ghoo one trust vision',
 u'self interested weak',
 u'receptionist',
 u'horrible',
 u'rude nasty',
 u'terrible',
 u'long wait colonoscopy',
 u'business man doctor',
 u'sooo dissapointed office staff',
 u'terrible surgery',
 u'waste time unconcerned doctor',
 u'first class surgeon',
 u'dr desai',
 u'horrific care',
 u'recommended',
 u'go',
 u'unprofessional judgmental',
 u'bad doctor',
 u'fabulous',
 u'irresponsible nurse staff',
 u'ineffective quack',
 u'love dr samdani team',
 u'great dr',
 u'courtesy bedside manners',
 u'stay away',
 u'good',
 u'expect courtesy information',
 u'coincidence chris writing time',
 u'call hospital show',
 u'gets',
 u'green authoritarian disrespectful',
 u'lost baby',
 u'almost died doctor',
 u'patient safety qualtiy care',
 u'reliable',
 u'type overconfident severe lack skill pride work',
 u'worst doctor ever mispleasure',
 u'bad doctor stay away',
 u'great place laser tattoo rem

In [34]:
print "Creating the bag of words...\n"
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 10000, min_df = 3) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

print train_data_features.shape

Creating the bag of words...

(1022, 185)


In [35]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
#print vocab

In [36]:
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print count, tag

3 agree
3 almost
8 amazing
9 another
3 anyone
4 appointment
9 arrogant
3 ask
3 attitude
15 avoid
21 away
6 awesome
10 awful
13 back
37 bad
22 bedside
29 best
6 better
13 beware
3 billing
3 breast
4 call
5 cancer
24 care
5 cares
13 caring
3 child
3 class
3 cocky
5 cold
3 communication
3 compassionate
4 completely
3 concern
4 customer
3 day
8 diagnosis
3 died
3 different
10 disappointed
4 disrespectful
5 doc
154 doctor
9 doctors
3 dont
84 dr
9 else
3 elsewhere
36 ever
13 excellent
38 experience
4 explain
5 extremely
3 family
10 find
4 first
7 follow
3 former
3 fraud
7 get
4 give
29 go
5 going
19 good
4 got
32 great
9 guy
6 happy
3 head
4 health
8 help
5 highly
49 horrible
5 hospital
3 important
4 impressed
4 insurance
3 interested
4 jerk
6 kaiser
3 keep
3 knee
5 knowledgeable
4 lack
3 lacks
5 life
7 like
7 listen
5 long
6 looking
5 love
6 man
3 manager
22 manner
6 manners
3 many
5 md
5 medical
3 medication
3 met
9 money
5 mother
3 mrs
3 much
3 nasty
3 need
9 needs
14 never
7 new
8 nice
3

In [37]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 50) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train['what_topics_is_this_review_about'])

forest

Training the random forest...


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [38]:
# Read the test data
#test = pd.read_csv("physicianTraining.csv", header=0, quoting=3, error_bad_lines=False, warn_bad_lines=False)

#test = test.drop_duplicates(subset='comment')
#test = test.dropna(subset=['comment'])

#test = test.reset_index()

test = finalTrain
print test.shape

# Create an empty list and append the clean reviews one by one
num_reviews = len(test['comment'])
clean_test_reviews = [] 

print "Cleaning and parsing the test set reviews...\n"
for i in xrange(0,num_reviews):
    if( (i+1) % 100 == 0 ):
        print "Review %d of %d\n" % (i+1, num_reviews)
    clean_review = review_to_words( test['comment'][i])
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], 'what_topics_is_this_review_about':result} )

(1022, 22)
Cleaning and parsing the test set reviews...

Review 100 of 1022

Review 200 of 1022

Review 300 of 1022

Review 400 of 1022

Review 500 of 1022

Review 600 of 1022

Review 700 of 1022

Review 800 of 1022

Review 900 of 1022

Review 1000 of 1022



In [39]:
import numpy as np

#f = open('results.txt', 'r+')

#test = test.replace(np.nan,' ', regex=True)
#output = output.replace(np.nan,' ', regex=True)

total = 0
correct = 0

for i in xrange(0, num_reviews):
    #f.write(test['comment'][i] + "\n")
    #f.write(test['what_topics_is_this_review_about'][i] + "\n")
    #f.write(output['what_topics_is_this_review_about'][i])
    #f.write('\n\n')
    total = total + 1
    if (test['what_topics_is_this_review_about'][i] == output['what_topics_is_this_review_about'][i]):
        correct = correct + 1

print str(float(correct)/float(total)) + "% accuracy on training data"

0.713307240705% accuracy on training data


In [40]:
test1 = finalTest
print test1.shape

# Create an empty list and append the clean reviews one by one
num_reviews1 = len(test1['comment'])
clean_test_reviews1 = [] 

print "Cleaning and parsing the test set reviews...\n"
for i in xrange(0,num_reviews1):
    if( (i+1) % 100 == 0 ):
        print "Review %d of %d\n" % (i+1, num_reviews1)
    clean_review1 = review_to_words( test1['comment'][i])
    clean_test_reviews1.append( clean_review1 )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features1 = vectorizer.transform(clean_test_reviews1)
test_data_features1 = test_data_features1.toarray()

# Use the random forest to make sentiment label predictions
result1 = forest.predict(test_data_features1)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output1 = pd.DataFrame( data={"id":test1["id"], 'what_topics_is_this_review_about':result1} )

(875, 22)
Cleaning and parsing the test set reviews...

Review 100 of 875

Review 200 of 875

Review 300 of 875

Review 400 of 875

Review 500 of 875

Review 600 of 875

Review 700 of 875

Review 800 of 875



In [41]:
total1 = 0
correct1 = 0

for i in xrange(0, num_reviews1):
    #total1 = total1 + 1
    if (test1['what_topics_is_this_review_about'][i] == output1['what_topics_is_this_review_about'][i]):
        #correct1 = correct1 + 1
        print "correctly guessed " + test1['what_topics_is_this_review_about'][i]
    elif (test1['what_topics_is_this_review_about'][i] in output1['what_topics_is_this_review_about'][i]):
        print "partially guessed " + test1['what_topics_is_this_review_about'][i] + " instead of " + output1['what_topics_is_this_review_about'][i]
    elif (output1['what_topics_is_this_review_about'][i] in test1['what_topics_is_this_review_about'][i]):
        print "partially guessed " + test1['what_topics_is_this_review_about'][i] + " instead of " + output1['what_topics_is_this_review_about'][i]
    else:
        print "incorrectly guessed " + test1['what_topics_is_this_review_about'][i] + " instead of " + output1['what_topics_is_this_review_about'][i]

#print str(float(correct1)/float(total1)) + "% accuracy on testing data"

incorrectly guessed bedside_manner instead of doctor_competence
correctly guessed bedside_manner
correctly guessed bedside_manner
incorrectly guessed bedside_manner instead of doctor_competence
incorrectly guessed bedside_manner instead of doctor_competence
incorrectly guessed wait_time instead of bedside_manner
incorrectly guessed other instead of bedside_manner
incorrectly guessed doctor_competence instead of bedside_manner
incorrectly guessed wait_time
staff instead of doctor_competence
likely_to_recommend
incorrectly guessed doctor_competence instead of wait_time
incorrectly guessed other instead of bedside_manner
correctly guessed bedside_manner
incorrectly guessed medical_history instead of doctor_competence
correctly guessed wait_time
partially guessed doctor_competence
bedside_manner instead of bedside_manner
partially guessed bedside_manner
staff instead of bedside_manner
incorrectly guessed bedside_manner instead of medical_history
incorrectly guessed responsiveness instead o

In [49]:
output1["what_topics_is_this_review_about"] = output1["what_topics_is_this_review_about"].str.split("\r")

test1["what_topics_is_this_review_about"] = test1["what_topics_is_this_review_about"].str.split("\r")

binarizer.fit()
actual = binarizer.transform(output1.what_topics_is_this_review_about)
predicted = binarizer.transform(test1.what_topics_is_this_review_about)


print test_target.size
print predicted.size
print actual.size

from sklearn.metrics import classification_report
print(classification_report(actual, predicted, target_names=list(binarizer.classes_)))



TypeError: 'numpy.float64' object is not iterable